<a href="https://colab.research.google.com/github/ShaunakSen/Deep-Learning/blob/master/Copy_of_2_1_ForwardAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1: Forward Mode Automatic Differentiation

Forward mode AD can simply be implemented by defining a class to represent [dual numbers](https://en.wikipedia.org/wiki/Dual_number) which hold the value and its derivative. The following skeleton defines a dual number and implements multiplication. 

__Tasks:__

- Addition (`__add__`) is incomplete - can you finish it? 
- Can you also implement division (`__truediv__`), subtraction (`__sub__`) and power (`__pow__`)?

In [13]:
import math

class DualNumber:
    def __init__(self, value, dvalue):
        self.value = value
        self.dvalue = dvalue

    def __str__(self):
        return str(self.value) + " + " + str(self.dvalue) + "ε"

    def __mul__(self, other):
        return DualNumber(self.value * other.value,
            self.dvalue * other.value + other.dvalue * self.value)
    
    def __add__(self, other):
        #TODO: finish me
        # YOUR CODE HERE
        return DualNumber(self.value + other.value, self.dvalue + other.dvalue)
      
    def __sub__(self, other):
        #TODO: finish me
        # YOUR CODE HERE
        return DualNumber(self.value - other.value, self.dvalue - other.dvalue)
      
    def __truediv__(self, other):
      other_conjugate = DualNumber(other.value, -other.dvalue)
      denominator = other * other_conjugate
      denominator_value = denominator.value
      numerator = self*other_conjugate
      print(numerator, denominator)
      
      result = DualNumber(numerator.value/denominator_value, numerator.dvalue/denominator_value)
      
      return result
    
    # TODO: add missing methods
    # YOUR CODE HERE
    # raise NotImplementedError()
    
print(DualNumber(4,4)/DualNumber(5,1))

20 + 16ε 25 + 0ε
0.8 + 0.64ε


In [14]:
# Tests

DualNumber(1,0) + DualNumber(1,0) / DualNumber(1,0) - DualNumber(1,0)**DualNumber(1,0)


1 + 0ε 1 + 0ε


TypeError: ignored

## Implementing math functions

We also need to implement some core math functions. Here's the sine function for a dual number:

In [16]:
def sin(x):
    return DualNumber(math.sin(x.value), math.cos(x.value)*x.dvalue)
print(DualNumber(4,3))
print(sin(DualNumber(4,3)))

4 + 3ε
-0.7568024953079282 + -1.960930862590836ε


__Task:__ can you implement the _cosine_ (`cos`), _tangent_ (`tan`), and _exponential_ (`exp`) functions in the code block below?

In [0]:
# TODO: implement additional math functions on dual numbers

def cos(x):
    # YOUR CODE HERE
    return DualNumber(math.cos(x.value), -math.sin(x.value)*x.dvalue)

def tan(x):
    # YOUR CODE HERE
    sec_sq = 1/(math.cos(x.value)**2)
    return DualNumber(math.tan(x.value), sec_sq*x.dvalue)

def exp(x):
    # YOUR CODE HERE
    return DualNumber(math.exp(x.value), math.exp(x.value))
    

In [0]:
# Tests
assert cos(DualNumber(0,0)).value == 1
assert tan(DualNumber(0,0)).value == 0
assert exp(DualNumber(0,0)).value == 1


## Time to try it out

We're now in a position to try our implementation.

__Task:__ 

- Try running the following code to compute the value of the function $z=x\cdot y+sin(x)$ given $x=0.5$ and $y=4.2$, together with the derivative $\partial z/\partial x$ at that point. 

In [18]:
# YOUR CODE HERE
# raise NotImplementedError()

x = DualNumber(0.5, 1)

y = DualNumber(4.2, 0)

sin_x = sin(x)

print(x*y + sin_x)

2.579425538604203 + 5.077582561890373ε


__Task__: Differentiate the above function with respect to $x$ and write the symbolic derivatives in the following box. Verify the result computed above is correct by plugging-in the values into your symbolic gradient expression.

YOUR ANSWER HERE

__Task:__ Now use the code block below to compute the derivative $\partial z/\partial y$ of the above expression (at the same point $x=0.5, y=4.2$ as above) and store the derivative in the variable `dzdy` (just the derivative, not the Dual Number). Verify by hand that the result is correct.

In [31]:
# YOUR CODE HERE
x = DualNumber(0.5, 0)

y = DualNumber(4.2, 1)


dzdy = (x*y + sin(x)).dvalue

print('dz/dy:', dzdy)

dz/dy: 0.5


In [0]:
#Tests
assert dzdy
assert type(dzdy) == float


__Task:__ Finally, use the code block below to experiment and test the other math functions and methods you created.

In [0]:
# YOUR CODE HERE
raise NotImplementedError()